In [2]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

## 1. Toy example on HumanDiseaseStr Table

**select * from human_disease_str limit 5;**

`
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
| uniprot_human | res_number | wildtype | mutant | pdb_id | chain_id | monomer_surface | ba_surface | id |
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
| ACTS_HUMAN    |        288 | D        | G      | 1t44   | A        |              68 |         68 |  1 |
| ACTS_HUMAN    |        294 | D        | V      | 1t44   | A        |              51 |         51 |  2 |
| ACTS_HUMAN    |        261 | E        | V      | 1t44   | A        |               0 |          0 |  3 |
| ACTS_HUMAN    |         17 | G        | R      | 1t44   | A        |            11.4 |       11.4 |  4 |
| ACTS_HUMAN    |        184 | G        | D      | 1t44   | A        |             4.4 |        4.4 |  5 |
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
5 rows in set (0.002 sec)
`

**select id,pdb_chain_id,db_code from pdb_chain_dbref_191022  where db_code = 'ACTS_HUMAN';**

`
+---------+--------------+------------+
| id      | pdb_chain_id | db_code    |
+---------+--------------+------------+
| 5558809 |       307702 | ACTS_HUMAN |
| 5765309 |      1126823 | ACTS_HUMAN |
| 5721478 |      1027804 | ACTS_HUMAN |
+---------+--------------+------------+
3 rows in set (0.104 sec)
`

**select  d.id,d.pdb_chain_id, v.uniprot_human, v.res_number, SUBSTR(a.seq, v.res_number, 10),v.wildtype,v.mutant,v.pdb_id from human_disease_str as v, dbref_uniprot_seq_200203 as d, astral_seq as a where v.uniprot_human = d.db_code and d.seq_id = a.id limit 10;**

`
+---------+--------------+---------------+------------+------------+----------+--------+--------+
| id      | pdb_chain_id | uniprot_human | res_number | substr     | wildtype | mutant | pdb_id |
+---------+--------------+---------------+------------+------------+----------+--------+--------+
| 5573071 |       340871 | EMD_HUMAN     |         54 | saassysfsd | S        | F      | 1jei   |
| 5573106 |       340949 | EMD_HUMAN     |         54 | saassysfsd | S        | F      | 1jei   |
| 5564562 |       321761 | EMD_HUMAN     |         54 | saassysfsd | S        | F      | 1jei   |
| 5563554 |       319867 | CYC_HUMAN     |         42 | gqapgysyta | G        | S      | 3nwv   |
| 5550326 |       280793 | FOXC2_HUMAN   |        125 | slnecfvkvp | S        | L      | 1d5v   |
| 5541682 |       257028 | FOXP2_HUMAN   |        553 | rhnlslhkcf | R        | H      | 2as5   |
| 5563491 |       319742 | GGA1_HUMAN    |        239 | gaaagssedl | G        | S      | 1oxz   |
| 5572417 |       339235 | GGA1_HUMAN    |        239 | gaaagssedl | G        | S      | 1oxz   |
| 5571888 |       337859 | GGA1_HUMAN    |        239 | gaaagssedl | G        | S      | 1oxz   |
| 5574367 |       344332 | GGA1_HUMAN    |        239 | gaaagssedl | G        | S      | 1oxz   |
+---------+--------------+---------------+------------+------------+----------+--------+--------+
`

### from the query above it seems the res_number completely matches the data and could perform a match


## 2. Design a relational table to store the variant data
 - Outer join took so long to execute :   
 `
 create table link_human_disease_str_dbref_uniprot_seq_200218 as SELECT v.id,d.id FROM human_disease_str as v
LEFT JOIN dbref_uniprot_seq_200203 as d ON v.uniprot_human = d.db_code
UNION
SELECT * FROM human_disease_str as v
RIGHT JOIN dbref_uniprot_seq_200203 as d ON v.uniprot_human = d.db_code limit 10
`
 - 1. **link_dbref_human_disease_str:** map reference and variant    
 `
 CREATE TABLE link_dbref_human_disease_str as select d.id as pdb_chain_dbref_id, v.id as variant_id from human_disease_str as v, dbref_uniprot_seq_200203 as d where v.uniprot_human = d.db_code`
 - 2. **human_disease_str:** variant table - store variant information with variant id
 - 3. if anything varies from version (pdb/uniprot/variant)

## 3. Inspection of the mapping between human_disease_str and dbref

#### number of variants mapped to the pdb_chain_dbref_id in human_disease_str

`
select count(distinct(variant_id)) from link_dbref_human_disease_str;
+-----------------------------+
| count(distinct(variant_id)) |
+-----------------------------+
|                        6024 |
+-----------------------------+
`

#### number of variants presented in human_disease_str
`
select count(distinct(id)) from human_disease_str;
+---------------------+
| count(distinct(id)) |
+---------------------+
|                6938 |
+---------------------+
`

#### inspect the remaining 914 variants from human_disease_str
`
select * from human_disease_str where id not in (select variant_id from link_dbref_human_disease_str) limit 5;
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
| uniprot_human | res_number | wildtype | mutant | pdb_id | chain_id | monomer_surface | ba_surface | id |
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
| AXN1_HUMAN    |        106 | L        | R      | 1dk8   | A        |               0 |          0 | 23 |
| BRAF1_HUMAN   |        594 | D        | G      | 3c4c   | A        |            21.2 |       21.2 | 29 |
| BRAF1_HUMAN   |        638 | D        | E      | 3c4c   | A        |             1.2 |        1.2 | 30 |
| BRAF1_HUMAN   |        501 | E        | G      | 3c4c   | A        |               3 |          3 | 31 |
| BRAF1_HUMAN   |        501 | E        | K      | 3c4c   | A        |               3 |          3 | 32 |
+---------------+------------+----------+--------+--------+----------+-----------------+------------+----+
`

## 4. Try resolving the 914 variants uniprot long_id from mapping accession number
- dbref_uniprot_seq_match_acc store mapping between db_accession and invalid uniprot identifiers
- pdb_chain_dbref_no_identifier contains the invalid identifiers collected when mapping dbref and uniprot    
`
mysql -h doppelbock scop -e "select * from human_disease_str where id not in (select variant_id from link_dbref_human_disease_str)" > unmatched_human_disease_str_200316.tsv`
`

In [165]:
missing_match = pd.read_csv('./missing_match_191209.tsv',sep = '\t')
invalid_dbref = pd.read_csv('./pdb_chain_dbref_no_identifier_191209.tsv',sep = '\t')
unmatched_hds = pd.read_csv('./unmatched_human_disease_str_200316.tsv',sep = '\t')

In [166]:
unmatched_hds.head()

,uniprot_human,res_number,wildtype,mutant,pdb_id,chain_id,monomer_surface,ba_surface,id
0,AXN1_HUMAN,106,L,R,1dk8,A,0.0,0.0,23
1,BRAF1_HUMAN,594,D,G,3c4c,A,21.2,21.2,29
2,BRAF1_HUMAN,638,D,E,3c4c,A,1.2,1.2,30
3,BRAF1_HUMAN,501,E,G,3c4c,A,3.0,3.0,31
4,BRAF1_HUMAN,501,E,K,3c4c,A,3.0,3.0,32


In [173]:
# unmatched Human Disease Str with valid_code map by invalid unip
missing_match_unmatched_hds = pd.merge(unmatched_hds,missing_match,left_on = "uniprot_human", right_on = "valid_code",how = "left" )


In [174]:
# missing_match_unmatched_hds.shape
missing_match_unmatched_hds.shape

(920, 11)

In [177]:
len(unmatched_hds["uniprot_human"].unique())

122

In [178]:
len(missing_match_unmatched_hds["uniprot_human"].unique())

122

In [194]:
len(missing_match_unmatched_hds.loc[~missing_match_unmatched_hds["db_accesion"].isnull(),]["uniprot_human"].unique())

22

In [195]:
still_missing_hds = missing_match_unmatched_hds.loc[missing_match_unmatched_hds["db_accesion"].isnull(),]
still_missing_hds



,uniprot_human,res_number,wildtype,mutant,pdb_id,chain_id,monomer_surface,ba_surface,id,db_accesion,valid_code
0,AXN1_HUMAN,106,L,R,1dk8,A,0.0,0.0,23,NaN,NaN
1,BRAF1_HUMAN,594,D,G,3c4c,A,21.2,21.2,29,NaN,NaN
2,BRAF1_HUMAN,638,D,E,3c4c,A,1.2,1.2,30,NaN,NaN
3,BRAF1_HUMAN,501,E,G,3c4c,A,3.0,3.0,31,NaN,NaN
4,BRAF1_HUMAN,501,E,K,3c4c,A,3.0,3.0,32,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
915,XRP2_HUMAN,118,R,L,3bh7,B,36.1,5.3,6931,NaN,NaN
916,ZIC3_HUMAN,253,C,S,2rpc,A,0.0,0.0,6935,NaN,NaN
917,ZIC3_HUMAN,286,H,R,2rpc,A,18.0,18.0,6936,NaN,NaN
918,ZIC3_HUMAN,323,T,M,2ej4,A,5.5,5.5,6937,NaN,NaN


## 0325

### Mapping unmatched variants uniprot name from db_accession resolved only 6 out of 122 from this set, the rest are simply not present in pdb_chain_dbref db_code column. 
- For now, try link through pdb_id to get the accession code through pypdb package

In [196]:
import pypdb
from bs4 import BeautifulSoup

In [198]:
all_info = pypdb.get_all_info('3h3g')
isinstance(all_info['polymer'], list)
all_info['polymer'][0]['macroMolecule']

[{'@name': 'Maltose/maltodextrin-binding periplasmic protein',
  'accession': {'@id': 'P0AEX9'}},
 {'@name': 'Parathyroid hormone/parathyroid hormone-related peptide receptor',
  'accession': {'@id': 'Q03431'}}]

In [203]:
def map_pdb_to_accession(pdb_id,chain_id):
    all_info = pypdb.get_all_info(pdb_id)
    if isinstance(all_info['polymer'], list):
         for polymer in all_info['polymer']:
                chains = get_chains(polymer)
                if chain_id in chains:
                    accession = get_accession(polymer['macroMolecule'])
    else:
        polymer = all_info['polymer']
        chains = get_chains(polymer)
        if chain_id in chains:
            accession = get_accession(polymer['macroMolecule'])
   
    return accession

def get_chains(polymer):
    if isinstance(polymer['chain'], list):
        chains = [chain['@id'] for chain in polymer['chain']]
    else:
        chains = [polymer['chain']['@id']]
    return chains

def get_accession(macro):
    if isinstance(macro, list):
        accession = [elem['accession']['@id'] for elem in macro][0]
    else:
        accession = macro['accession']['@id']
    return accession

In [204]:
map_pdb_to_accession('3h3g','A')

'P0AEX9'

In [209]:
accs = []

In [210]:
for i in range(100):
    
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)

1dk8
A
O15169
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
3c4c
A
P15056
1ygr
A
P08575
2ojw
A
P15104
2ojw
A
P15104
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
3hhm
A
P42336
2qcf
A
P11172
2wns
A
P11172
2wns
A
P11172
1x4a
A
Q07955
2e1q
A
P47989
2e1q
A
P47989
2e1q
A
P47989
2uxw
A
P49748
2uxw
A
P49748
2uxw
A
P49748
3b96
A
P49748
2uxw
A
P49748
3b96
A
P49748
2uxw
A
P49748
2uxw
A
P49748
2uxw
A
P49748
2uxw
A
P49748
2uxw
A
P49748
2uxw
A
P49748
3b96
A
P49748
2uxw
A

In [212]:
for i in range(100,217):
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)



3ghm
A
Q76LX8
3ghn
A
Q76LX8
3ghm
A
Q76LX8
3ghn
A
Q76LX8
3ghn
A
Q76LX8
2vxp
A
Q15582
2vxp
A
Q15582
2vxp
A
Q15582
1x3b
A
Q15582
1x3b
A
Q15582
2vxp
A
Q15582
2vxp
A
Q15582
1x3b
A
Q15582
2vxp
A
Q15582
2vxp
A
Q15582
1x3b
A
Q15582
2vxp
A
Q15582
2vxp
A
Q15582
1x3b
A
Q15582
1x3b
A
Q15582
1x3b
A
Q15582
1x3b
A
Q15582
1x3b
A
Q15582
2vaf
A
O14958
2vaf
A
O14958
2vaf
A
O14958
2nsm
A
P15169
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1dc2
A
P42771
1bi7
B
P42771
1dc2
A
P42771
1dc2
A
P42771
1dc2
A
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1dc2
A
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B
P42771
1bi7
B

TypeError: 'NoneType' object is not subscriptable

In [215]:
for i in range(217,still_missing_hds.shape[0]):   
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)
    




2fy4
A
P28329
2fy4
A
P28329
2v53
C
P02461
2v53
D
P02461
2v53
B
P02461
3dmw
C
P02461
3dmw
C
P02461
3dmw
C
P02461
3dmw
B
P02461
3dmw
A
P02461
3dmw
A
P02461
3dmw
A
P02461
3dmw
A
P02461
3dmw
B
P02461
3dmw
A
P02461
3dmw
C
P02461
3dmw
A
P02461
3dmw
A
P02461
3dmw
A
P02461
3dmw
A
P02461
1vzj
I
Q9Y215
3fby
A
P49747
3fby
A
P49747
3fby
B
P49747
3fby
A
P49747
3fby
C
P49747
3fby
A
P49747
3fby
A
P49747
3fby
B
P49747
3fby
C
P49747
3fby
C
P49747
3fby
B
P49747
3fby
A
P49747
3fby
A
P49747
3fby
B
P49747
3fby
B
P49747
3fby
B
P49747
3fby
C
P49747
3fby
C
P49747
3fby
C
P49747
3fby
C
P49747
3fby
B
P49747
3fby
B
P49747
3fby
A
P49747
3fby
B
P49747
3fby
B
P49747
3eqm
A
P11511
3eqm
A
P11511
3eqm
A
P11511
3eqm
A
P11511
2y1z
B
P02511
2e6w
A
Q9Y2W7
2e6w
A
Q9Y2W7
2zw3
C
P29033
2zw3
F
P29033
3iz1
A
P29033
2zw3
B
P29033
2zw3
D
P29033
2zw3
D
P29033
2zw3
F
P29033
3iz1
A
P29033
2zw3
E
P29033
2zw3
F
P29033
2zw3
F
P29033
2zw3
F
P29033
3iz2
C
P29033
2zw3
E
P29033
2zw3
E
P29033
2zw3
F
P29033
2zw3
A
P29033
2zw3
A
P29033
2zw3
A

TypeError: 'NoneType' object is not subscriptable

In [217]:
for i in range(317,still_missing_hds.shape[0]):   
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)
    
    


2fdb
M
P55075
1wsv
B
P48728
1wsv
B
P48728
1wsv
B
P48728
1wsr
B
P48728
1wsv
B
P48728
1wsr
B
P48728
1wsv
A
P48728
3v70
A
Q8WWP7
2acx
A
P43250
3nyn
A
P43250
3ecr
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3eq1
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3ecr
A
P08397
3eq1
A
P08397
3eq1
A

TypeError: 'NoneType' object is not subscriptable

In [219]:
for i in range(417,still_missing_hds.shape[0]):   
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)

3mp4
B
P35914
3mp3
B
P35914
3mp3
B
P35914
3mp3
B
P35914
3mp3
A
P35914
2cw6
A
P35914
2cw6
C
P35914
3mp4
A
P35914
3mp5
F
P35914
2cw6
A
P35914
3mp4
B
P35914
2cw6
A
P35914
3mp5
E
P35914
2cw6
A
P35914
2cw6
A
P35914
3mp3
B
P35914
2cw6
E
P35914
2pe4
A
Q12794
2pmv
C
P27352
2h3n
D
P15814
3up1
B
P16871
3di2
B
P16871
3up1
B
P16871
2h8b
A
P51460
2kb9
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2kb9
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2vj2
A
P78504
2c9y
A
P54819
2c9y
A
P54819
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
1zlg
A
P23352
2w4o
A
Q16566
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
2k21
A
P15382
3hfe
A
P51787
3bj4
A
P51787
3bj4
A
P51787
3bj4
B
P51787
2hlz
A
P50053
2hlz
A
P50053
2i0e
A
P05771
3e9k
A
Q16719
3e0g
A
P42702
2wa0
A
P43358
2vrg
A
Q8NI22
2vrg
A
Q8NI22
3pa6
A

TypeError: 'NoneType' object is not subscriptable

In [221]:
for i in range(517,still_missing_hds.shape[0]):   
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)

2o8e
A
P43246
3thx
A
P43246
2o8e
A
P43246
3thy
A
P43246
3thy
A
P43246
3thy
A
P43246
3thy
A
P43246
3thy
A
P43246
2o8f
A
P43246
2o8b
A
P43246
3thy
A
P43246
3thz
A
P43246
3thw
A
P43246
3thw
A
P43246
2o8b
A
P43246
3thz
A
P43246
2o8b
A
P43246
3thx
A
P43246
2o8b
A
P43246
2o8b
A
P43246
2o8c
A
P43246
3thx
A
P43246
3thy
A
P43246
2o8b
A
P43246
3thx
A
P43246
3thx
A
P43246
2o8e
A
P43246
2o8b
A
P43246
2o8e
A
P43246
2o8e
A
P43246
3thx
A
P43246
3thz
A
P43246
2o8b
A
P43246
3thx
A
P43246
2o8d
A
P43246
2o8d
A
P43246
3thy
A
P43246
3thz
A
P43246
3thx
A
P43246
2o8b
A
P43246
2o8b
A
P43246
2o8b
A
P43246
2o8d
A
P43246
2o8f
A
P43246
3thx
A
P43246
2o8b
A
P43246
3thw
A
P43246
3thw
A
P43246
3thx
A
P43246
2o8b
A
P43246
3thw
A
P43246
2o8d
A
P43246
3thz
A
P43246
2o8e
B
P52701
2o8b
B
P52701
2o8c
B
P52701
2o8d
B
P52701
2o8f
B
P52701
2o8b
B
P52701
2o8e
B
P52701
3ggf
A
Q9P289
2qtz
A
Q9UBK8
2qtl
A
Q9UBK8
2qtl
A
Q9UBK8
2xiq
A
P22033
3bic
A
P22033
2xij
A
P22033
2xij
A
P22033
2xij
A
P22033
2xiq
B
P22033
2xij
A
P22033
2xiq
B

TypeError: 'NoneType' object is not subscriptable

In [223]:
for i in range(617,still_missing_hds.shape[0]):   
    pdb_id = still_missing_hds.iloc[i,4]
    chain_id = still_missing_hds.iloc[i,5]
    print(pdb_id)
    print(chain_id)
    acc = map_pdb_to_accession(pdb_id,chain_id)
    print(acc)
    accs.append(acc)

2xiq
A
P22033
3bic
B
P22033
2xiq
B
P22033
2xij
A
P22033
2xiq
A
P22033
2xij
A
P22033
2xij
A
P22033
2xiq
A
P22033
2xiq
A
P22033
2xij
A
P22033
2xij
A
P22033
2xiq
B
P22033
2xiq
A
P22033
3bic
B
P22033
3bic
B
P22033
2xij
A
P22033
2xij
A
P22033
2xij
A
P22033
2xij
A
P22033
2xij
A
P22033
2xiq
A
P22033
2xij
A
P22033
3bic
B
P22033
2xiq
B
P22033
2xiq
B
P22033
2xiq
A
P22033
3bic
B
P22033
3bic
B
P22033
3bic
B
P22033
3bic
A
P22033
2xiq
B
P22033
2xiq
B
P22033
2xiq
B
P22033
2xiq
A
P22033
3bic
B
P22033
3bic
A
P22033
2xiq
A
P22033
2xiq
B
P22033
2xiq
A
P22033
2xiq
A
P22033
2xiq
A
P22033
2xiq
A
P22033
3bic
B
P22033
2xij
A
P22033
2xiq
B
P22033
2xij
A
P22033
2xij
A
P22033
2xij
A
P22033
3dgr
A
P43490
2jvx
A
Q9Y6K9
2jvx
A
Q9Y6K9
2jvx
A
Q9Y6K9
2jvy
A
Q9Y6K9
2jvx
A
Q9Y6K9
3brv
D
Q9Y6K9
3cl3
E
Q9Y6K9
3fx0
A
Q9Y6K9
3fx0
B
Q9Y6K9
3fx0
B
Q9Y6K9
3fx0
B
Q9Y6K9
3fx0
B
Q9Y6K9
3f02
B
Q99574
3f02
B
Q99574
3sr2
H
Q9H9Q4
3rwr
W
Q9H9Q4
3gkh
A
O15118
3gkh
A
O15118
3gkh
A
O15118
3gki
A
O15118
3gkh
A
O15118
3gkh
A
O15118
3gki
A

In [224]:
len(accs)

817

In [214]:
still_missing_hds.shape[0]

817

In [228]:
missing_match_unmatched_hds.loc[missing_match_unmatched_hds["db_accesion"].isnull(),'db_accesion'] = accs

In [231]:
any(missing_match_unmatched_hds['db_accesion'].isnull())

False

## All variants in roland mapped with a db_accession code by pdb chain and pdb entry identifier

In [232]:
# The resolved version
missing_match_unmatched_hds.to_csv("missing_match_unmatched_hds_190325.csv")

## TO-DO
1. import the csv file into doppelbock and then inner than uniprot_accession with the db_accession column in missing_match_unmatched_hds
2. Check if all the variants now have a valid uniprot identifier

In [7]:
missing_match_unmatched_hds = pd.read_csv("missing_match_unmatched_hds_190325.csv")

In [8]:
missing_match_unmatched_hds = missing_match_unmatched_hds.loc[:,["uniprot_human","pdb_id","chain_id","db_accesion"]]

In [15]:
missing_match_unmatched_hds.to_csv("missing_match_unmatched_hds_190331.csv",index = False)

In [14]:
missing_match_unmatched_hds

,uniprot_human,pdb_id,chain_id,db_accesion
0,AXN1_HUMAN,1dk8,A,O15169
1,BRAF1_HUMAN,3c4c,A,P15056
2,BRAF1_HUMAN,3c4c,A,P15056
3,BRAF1_HUMAN,3c4c,A,P15056
4,BRAF1_HUMAN,3c4c,A,P15056
...,...,...,...,...
915,XRP2_HUMAN,3bh7,B,O75695
916,ZIC3_HUMAN,2rpc,A,O60481
917,ZIC3_HUMAN,2rpc,A,O60481
918,ZIC3_HUMAN,2ej4,A,O60481


## Monday March 30th

## import missing_match_unmatched_hds_190331 into doppelbock

`
CREATE TABLE missing_match_unmatched_hds_190331(
    uniprot_human TEXT NOT NULL,
    pdb_id TEXT NOT NULL,
    chain_id varchar(1) NOT NULL,
    db_accession TEXT NOT NULL
);
`

`
LOAD DATA INFILE '/mnt/net/ipa.jmcnet/data/h/ych323/tmp/missing_match_unmatched_hds_190331.csv' 
INTO TABLE missing_match_unmatched_hds_190331 
COLUMNS TERMINATED BY ','
OPTIONALLY ENCLOSED BY '"'
ESCAPED BY '"'
LINES TERMINATED BY '\n'
IGNORE 1 LINES;
`



## using missing_match_unmatched_hds_190331 to resolve the mapping
- actually, the previous mapping scheme solely through uniprot human/ uniprot accession code may actually not be accurate, since each uniprot identifier could correspond to multiple pdb chains, and so may not be the correct pdb_chain_dbref being mapped to.

1) for each pdb_id and chain_id provided by the variant data set, select the corresponding pdb_chain_id from the pdb_chain table (the pdb_chain table only contains the pdb_release_id, and so a column containing the pdb_id is needed to be added to pdb_chain_table)  

2) for each of the pdb_chain_id selected, get the pdb_chain_dbref for the pdb_chain_id, and naturally the corresponding uniprot accession in the pdb_chain_dbref table

3) using the accession_id, map from uniprot_accession to the valid uniprot identifier

4) using the uniprot identifier, query the corresponding sequence

5) using the sequence and the index of the residue being modified, query out the official uniprot residues from the functions

6) build a table with variant_id, pdb_id, chain_id, pdb_chain_id,pdb_chain_dbref_id, uniprot_accession, long_id, uniprot_id --> extracted sequences

**caught an error in previous update on pdb_chain_dbref_191022 **

`
drop table pdb_chain_dbref_191022
`

`
create table pdb_chain_dbref_191022 as select d.* from pdb_chain_dbref as d, raf_version_3_release_18 as r where d.db_name = 'UNP' and d.pdb_align_start is not NULL and d.pdb_align_end is not NULL and d.db_align_start > 0 and d.db_align_end > 0 and d.pdb_chain_id = r.pdb_chain_id;
`

`
select SUBSTRING(SUBSTRING(pdb_path,42,43),1,4) from pdb_local where pdb_release_id = 149865;
+------------------------------------------+
| SUBSTRING(SUBSTRING(pdb_path,42,43),1,4) |
+------------------------------------------+
| 2fb8                                     |
+------------------------------------------+
`

## 200403

0. pdb_chain_dbref_191022 properties:
   - align to 'UNP'
   - has only pdb_chain_id in the raf_version_3_release_18 table
   - valid db_align_start, db_align_end, pdb_align_start and pdb_align_end 
   - db_align for uniprot is 1-indexed so both db_align_start and db_align_end >0
   - no NULL in pdb_align_start and pdb_align_end 
   - no length difference between db_align_end-db_align_start and pdb_align_end - pdb_align_start
   
1. building a pdb_chain_200405: A table containing the pdb_chain table with all the entries of pdb_chain_id in pdb_chain_dbref_191022 and with an extra column with the pdb_id attached 


    1. distinct_pdb_chain_id_200405 contains the unique pdb_chain_id in pdb_chain_dbref_191022
`
CREATE TABLE distinct_pdb_chain_id_200405 AS
select distinct(pdb_chain_id) as pdb_chain_id from pdb_chain_dbref_191022
`

`  
CREATE TABLE pdb_chain_200405 AS SELECT chain.*,l.pdb_path as pdb_path FROM pdb_chain as chain, pdb_local as l,distinct_pdb_chain_id_200405 as d WHERE chain.id = d.pdb_chain_id AND chain.pdb_release_id = l.pdb_release_id;
`


`
ALTER TABLE pdb_chain_200405 ADD pdb_id TEXT NOT NULL;
UPDATE pdb_chain_200405 SET pdb_id = SUBSTRING(SUBSTRING(pdb_path,42,43),1,4);
`



## 200404

1. Check if pdb_chain_200405 covers the entire set of pdb_id in human_disease_str

`
select count(distinct(pdb_id)) from human_disease_str;
+-------------------------+
| count(distinct(pdb_id)) |
+-------------------------+
|                    1343 |
+-------------------------+
`

`
select count(distinct(pdb_id)) from pdb_chain_200405;
+-------------------------+
| count(distinct(pdb_id)) |
+-------------------------+
|                   83045 |
+-------------------------+
`

`
select count(distinct(hds.pdb_id)) from pdb_chain_200405 as c,human_disease_str as hds where hds.pdb_id = c.pdb_id;
+-----------------------------+
| count(distinct(hds.pdb_id)) |
+-----------------------------+
|                        1071 |
+-----------------------------+
`



## 190406
1. to expand the pdb_chain_id to cover the full release = 18, re-build the distinct_pdb_chain_id_200405 table

`
CREATE TABLE distinct_pdb_chain_id_200405 AS
select distinct(pdb_chain_id) as pdb_chain_id from raf_version_3_release_18
`

`  
CREATE TABLE pdb_chain_200405 AS SELECT chain.*,l.pdb_path as pdb_path FROM pdb_chain as chain, pdb_local as l,distinct_pdb_chain_id_200405 as d WHERE chain.id = d.pdb_chain_id AND chain.pdb_release_id = l.pdb_release_id;
`

`
ALTER TABLE pdb_chain_200405 ADD pdb_id TEXT NOT NULL;
UPDATE pdb_chain_200405 SET pdb_id = SUBSTRING(SUBSTRING(pdb_path,42,43),1,4);
`

`
select count(distinct(hds.pdb_id)) from pdb_chain_200405 as c,human_disease_str as hds where hds.pdb_id = c.pdb_id;
+-----------------------------+
| count(distinct(hds.pdb_id)) |
+-----------------------------+
|                        1088 |
+-----------------------------+
`

2. Check if all the accession mapped are inside uniprot_accession table

`
create table inspect_missing_match_unmatched_hds_1900406 as select missing_match_unmatched_hds_190331 as m, uniprot_accession as u where m.db_accession = u.accession
`

`
drop table inspect_missing_match_unmatched_hds_1900406;
`

3. Catch a bug in generating the uniprot entries: when there're multiple lines of AC, only the last line of accession is inserted in.

## 190408
1. Fix the bug in MakeDBUniprot: when there're multiple lines of AC, only the last line of accession is inserted in.

`
select distinct(db_accession) from missing_match_unmatched_hds_190331 where db_accession not in (select db_accession from inspect_missing_match_unmatched_hds_1900406);
+--------------+
| db_accession |
+--------------+
| P15056       |
| P11172       |
| Q15582       |
| P02461       |
| Q9Y215       |
| P29033       |
| Q9BUP3       |
| O60520       |
| Q12794       |
| P16871       |
| P54819       |
| P51787       |
| P05771       |
| P00387       |
| Q01968       |
| Q5BJD7       |
| P54886       |
| P12955       |
| O60260       |
+--------------+
19 rows in set (0.355 sec)
`

`
MySQL [scop]> select * from uniprot where long_id = 'BRAF_HUMAN';
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| 35220769 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           1 |
| 92041473 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           0 |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
2 rows in set (0.042 sec)
`

`
MySQL [scop]> select * from uniprot_accession where uniprot_id = 35220769;
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   35220769 | Q13878    |
|   35220769 | Q3MIN6    |
|   35220769 | Q9UDP8    |
|   35220769 | Q9Y6T3    |
+------------+-----------+
4 rows in set (0.026 sec)
`

`
MySQL [scop]> select * from pdb_chain_uniprot limit 4;
+--------------+-------------------+
| pdb_chain_id | uniprot_accession |
+--------------+-------------------+
|       396709 | Q72KX2            |
|       396710 | Q72KX2            |
|       396711 | Q72KX2            |
|       396706 | Q72KX2            |
+--------------+-------------------+
4 rows in set (0.002 sec)
`

`
MySQL [scop]> select * from uniprot_accession where accession = 'Q72KX2';
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   92070166 | Q72KX2    |
+------------+-----------+
1 row in set (0.016 sec)
`

## 190410
The code is used to update uniprot_accession table for all the uniprot entries
in the most recent uniprot release

1. iterate through each entry in the most recent uniprot release 
2. update the accession information only in uniprot_accession table
3. if the entry is not in current uniprot, update entire information (which should not happen)

4. build a 200405.dat that previously has issue with the second row Q13878; Q3MIN6; Q9UDP8; Q9Y6T3;

`javac MakeDBUniprotAccession.java`

`java gov.lbl.scop.app.MakeDBUniprotAccession 200405.dat`

## 190412

`
select * from uniprot where long_id = 'BRAF_HUMAN'
    -> ;
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| 35220769 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           1 |
| 92041473 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           0 |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
`

### The old code only contains the last row of accessions

`
select * from uniprot_accession where uniprot_id = 92041473;
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   92041473 | Q13878    |
|   92041473 | Q3MIN6    |
|   92041473 | Q9UDP8    |
|   92041473 | Q9Y6T3    |
+------------+-----------+
4 rows in set (0.009 sec)
`

### The new fixed (running MakeDBUniprotAccession.java) has the following

`
select * from uniprot_accession where uniprot_id = 92041473;
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   92041473 | P15056    |
|   92041473 | A4D1T4    |
|   92041473 | B6HY61    |
|   92041473 | B6HY62    |
|   92041473 | B6HY63    |
|   92041473 | B6HY64    |
|   92041473 | B6HY65    |
|   92041473 | B6HY66    |
|   92041473 | Q13878    |
|   92041473 | Q3MIN6    |
|   92041473 | Q9UDP8    |
|   92041473 | Q9Y6T3    |
+------------+-----------+
12 rows in set (0.002 sec)
`

### before running, grab some stats on the old uniprot_accession table
`
select count(*) from uniprot_accession;
+----------+
| count(*) |
+----------+
| 82702239 |
+----------+
`

### now iterate through the 2 dat files in the latest uniprot release on a copy of the uniprot_accession table named uniprot_accession_200412
`java gov.lbl.scop.app.MakeDBUniprot /lab/db/uniprot/20190410/uniprot_sprot.dat`
`java gov.lbl.scop.app.MakeDBUniprot /lab/db/uniprot/20190410/uniprot_trembl.dat`

## 190414
- Catching the issue relating to obsoleteness


ID   BRAF_COTJA              Reviewed;         807 AA.

AC   P34908;

DT   01-FEB-1994, integrated into UniProtKB/Swiss-Prot.

DT   01-FEB-1994, sequence version 1.

DT   16-JAN-2019, entry version 120.

DE   RecName: Full=Serine/threonine-protein kinase B-raf;

DE            EC=2.7.11.1;

DE   AltName: Full=Proto-oncogene B-Raf;

DE   AltName: Full=Proto-oncogene c-Rmil;

DE   AltName: Full=Serine/threonine-protein kinase Rmil;

GN   Name=BRAF; Synonyms=RMIL;



`
select * from uniprot where long_id = 'BRAF_COTJA';

+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+

| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |

+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+

| 35220768 | BRAF_COTJA |            1 |           1 | 1994-02-01 | Serine/threonine-protein kinase B-raf |           1 |

+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
`



- confused on why the entry in the latest release is classified as is_obsolete = 1, and so I found the following line in MakeDBUniprot.java did so:



`
if (skipLines==0)

                stmt.executeUpdate("update uniprot set is_obsolete=1 where is_swissprot="+(isSprot ? 1 : 0));
`

- As for mapping with variants I only used the  is_obsolete=0 entries in uniprot table, and wondering is this a bug in the previous code

- A side note that is although the above line is carried in the makedbuniprot.java before I touch and execute the code, I commented it out when running my svn version to import uniprot entries not in pdb_chain_dbref, and so there are obsolete = 0 entries if they 



### 190416
- Following John-Marc's feedback on the obsoleteness, I think it's the best to reperform importing latest uniprot entry, and may worth just to create a new uniprot data base to see how the accession only based scheme perform 
- Add functionality to update and curate the is_obsolete column as well


190419
- finish implementing a new function MakeDBUniprot_200416.java

- The is_obsolete issue is fixed as the results below show:
`
select * from uniprot where long_id = 'BRAF_COTJA';
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| 35220768 | BRAF_COTJA |            1 |           1 | 1994-02-01 | Serine/threonine-protein kinase B-raf |           0 |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
`

- The db_accession problem is fixed as below shown:     

`
select * from uniprot where long_id = 'BRAF_HUMAN';
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| 35220769 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           0 |
| 92041473 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           0 |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
2 rows in set (0.002 sec)
`

`
MySQL [scop]> select * from uniprot_accession where uniprot_id = 35220769;
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   35220769 | P15056    |
|   35220769 | A4D1T4    |
|   35220769 | B6HY61    |
|   35220769 | B6HY62    |
|   35220769 | B6HY63    |
|   35220769 | B6HY64    |
|   35220769 | B6HY65    |
|   35220769 | B6HY66    |
|   35220769 | Q13878    |
|   35220769 | Q3MIN6    |
|   35220769 | Q9UDP8    |
|   35220769 | Q9Y6T3    |
+------------+-----------+
`

## 190420
- run MakeDBUniprot_200416.java on both sprot and trembl.dat of 190409 release of uniprot

`
javac MakeDBUniprot_200416.java
`

`java gov.lbl.scop.app.MakeDBUniprot_200416 /lab/db/uniprot/20190410/uniprot_sprot.dat`

- grab stats after running on uniprot_sprot.dat

`
select count(*) from uniprot where is_swissprot = 1 and is_obsolete = 0;
+----------+
| count(*) |
+----------+
|   559634 |
+----------+
`

`
select count(distinct(long_id)) from uniprot where is_swissprot = 1 and is_obsolete = 0;
+--------------------------+
| count(distinct(long_id)) |
+--------------------------+
|                   559634 |
+--------------------------+
1 row in set (26.188 sec)
`

Thus, it is confirmed that every entries in swiss_prot has one and only one entry in uniprot table.

- Now, re-confirm that the accession is fixed by 

`
select * from uniprot where long_id = 'BRAF_HUMAN';
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| id       | long_id    | is_swissprot | seq_version | seq_date   | entry_name                            | is_obsolete |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
| 35220769 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           0 |
| 92041473 | BRAF_HUMAN |            1 |           4 | 2004-07-19 | Serine/threonine-protein kinase B-raf |           1 |
+----------+------------+--------------+-------------+------------+---------------------------------------+-------------+
2 rows in set (0.003 sec)
`

`
MySQL [scop]> select * from uniprot_accession where uniprot_id = 35220769;
+------------+-----------+
| uniprot_id | accession |
+------------+-----------+
|   35220769 | P15056    |
|   35220769 | A4D1T4    |
|   35220769 | B6HY61    |
|   35220769 | B6HY62    |
|   35220769 | B6HY63    |
|   35220769 | B6HY64    |
|   35220769 | B6HY65    |
|   35220769 | B6HY66    |
|   35220769 | Q13878    |
|   35220769 | Q3MIN6    |
|   35220769 | Q9UDP8    |
|   35220769 | Q9Y6T3    |
+------------+-----------+
12 rows in set (0.003 sec)
`

`java gov.lbl.scop.app.MakeDBUniprot_200416 /lab/db/uniprot/20190410/uniprot_trembl.dat`

- Further customize and finalize the visualizer by adding in variant information section and instructure to color/de-select variants.

## 200421
### TO-DO:
    1. After inspecting no issue with the new uniprot table, check whether all the accession code in corrected uniprot accession table could be resolved for HuVarBase_mis_small.
    2. If so, re-merge the pdb_chain_dbref_19... with the uniprot table and pdb_chain table (with only the non-obsolete entries) using db_accession instead of db_code, one way to approach is to iterate through db_ref entries and query first uniprot_accession then the corresponding uniprot table. 
    3. If so, proceed down by establishing a match between human_disease_str ---> dbref ----> uniprot seq and raf seq

- The uniprot_trembl.dat is still updating
Meanwhile, write out executions needed to be done after MakeDBUniprot__200416 finished to run

    1. Create the mapping between chain_id,pdb_ib,uniprot,pdb_chain_dbref,uniprot_accession, uniprot_seq, and astral_seq
        - (chain_id and pdb_id should be included in addition to the pdb_chain_id to aid the merging of chain-specific variants)
    2. Name this new table dbref_uniprot_seq, drop the old one
    3. mapping the variant and dbref using accession

- Instead of making pypdb run on server and update the variant table to have the accession column for Roland database, select directly from Huvarbase a subset of missense mutation to test for the corrected accession entries in uniprot

`
select count(*) from huvarbase where pdb_id != '-' and uniprot_id != '-';
+----------+
| count(*) |
+----------+
|   127080 |
+----------+
`

`
select count(*) from huvarbase where pdb_id != '-' and mutation_type = 'Missense';
+----------+
| count(*) |
+----------+
|   114909 |
+----------+
`

- for now, select 10000 out of 114909 missense entries in HuVarBase as a test set **huvarbase_mis_small**

`
create table huvarbase_mis_small as select * from huvarbase where pdb_id != '-' and mutation_type = 'Missense' ORDER BY RAND() limit 10000;
`

`
select count(*) from huvarbase_mis_small where pdb_id != '-' and uniprot_id != '-';
+----------+
| count(*) |
+----------+
|    10000 |
+----------+
`

### Add new features into the webpage:

`
describe huvarbase_mis_small;
+----------------------+--------------+------+-----+---------+-------+
| Field                | Type         | Null | Key | Default | Extra |
+----------------------+--------------+------+-----+---------+-------+
| gene_name            | varchar(100) | YES  |     | NULL    |       |
| chromo_coordin       | varchar(100) | YES  |     | NULL    |       |
| DNA_mutation         | varchar(100) | YES  |     | NULL    |       |
| mutation_type        | varchar(100) | YES  |     | NULL    |       |
| orgin                | varchar(100) | YES  |     | NULL    |       |
| rs_id                | varchar(100) | YES  |     | NULL    |       |
| protein_change       | varchar(100) | YES  |     | NULL    |       |
| uniprot_id           | varchar(100) | YES  |     | NULL    |       |
| pdb_id               | varchar(100) | YES  |     | NULL    |       |
| isoform_canoncial_id | varchar(100) | YES  |     | NULL    |       |
| neighboring_res      | varchar(100) | YES  |     | NULL    |       |
| sec_structure        | varchar(100) | YES  |     | NULL    |       |
| conserv_score        | double       | YES  |     | NULL    |       |
| function             | varchar(100) | YES  |     | NULL    |       |
| domain               | varchar(100) | YES  |     | NULL    |       |
| subcell_locali       | varchar(100) | YES  |     | NULL    |       |
| post_trans_modif     | varchar(100) | YES  |     | NULL    |       |
| disease              | varchar(100) | YES  |     | NULL    |       |
| disease_class        | varchar(100) | YES  |     | NULL    |       |
| ref_db               | varchar(100) | YES  |     | NULL    |       |
| id                   | int(11)      | NO   |     | 0       |       |
+----------------------+--------------+------+-----+---------+-------+
`